<img src='../img/logo.png' alt='DS Market logo' height='150px'>

# 2 - Feature Engineering

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)
* [D. Creating a master dataframe](#master_df)
* [E. Creating aggregated dataframes](#master_df)


## A. Introduction <a class="anchor" id="introduction"></a>

In the following notebook, we will be generating some features that will be needed for the analysis and the future models to generate.

Disclaimer: Running this dataframe requires quite some time (up to 30 - 40 minutes, depending on your computer). Whenever possible, download the file from the following [GDrive link](#https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing).

## B. Importing Libraries <a class="anchor" id="libraries"></a>

In [1]:
# system and path management
import sys
sys.path.append('../scripts') # including helper functions inside the scripts folder

# removing system warnings
import warnings
warnings.filterwarnings('ignore')

# data manipulation
import pandas as pd
import numpy as np

# helper functions
import file_management

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.2f}'.format

## C. Importing Data <a class="anchor" id="data"></a>

In [2]:
# downloading the processed data files from gdrive, in case these were not available
directory = '../data/processed/'
urls = [
    {'filename': 'sales_processed.csv', 'url': 'https://drive.google.com/file/d/1JdeAgraKcaFQJrjG2HPVb5D0VD0iTlNB/view?usp=sharing'},
    {'filename': 'prices_processed.csv', 'url': 'https://drive.google.com/file/d/1pSEJAQfAU-owDjKmxcPrxf3CpGFivwa6/view?usp=sharing'},
    {'filename': 'calendar_processed.csv', 'url': 'https://drive.google.com/file/d/1Lnji96iBkTpFiWo-QXeW3TvESiNYWCML/view?usp=sharing'}
]
        
file_management.download_files_from_url(urls, directory)        

sales_processed.csv file already exists in ../data/processed/
prices_processed.csv file already exists in ../data/processed/
calendar_processed.csv file already exists in ../data/processed/


['./../data/processed//sales_processed.csv',
 './../data/processed//prices_processed.csv',
 './../data/processed//calendar_processed.csv']

In [3]:
sales = pd.read_csv(directory + 'sales_processed.csv', index_col = 0)
prices = pd.read_csv(directory + 'prices_processed.csv', index_col = 0)
calendar = pd.read_csv(directory + 'calendar_processed.csv', index_col = 0)

## D. Creating a master dataframe <a class="anchor" id="master_df"></a>

In [ ]:
# generating a dataframe with each row being the total amount of sales per day with each product
master = sales.melt(
        id_vars = ['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], 
        var_name = 'd', 
        value_name = 'num_sales'
)
master

In [ ]:
# removing unnecessary columns that can be easily regenerated with the id or with a simple dictionary ('store')
master.drop(columns = ['item', 'category', 'department', 'store_code', 'region', 'store'], inplace = True)
master

In [ ]:
# merging the master sales with calendar in order to get the date translation between 'd' and the real 'date'
master = pd.merge(
    master,
    calendar,
    on = 'd'
)

master

In [ ]:
# master dataframe sorting, cleanup and feature generation for other merges
master.sort_values(by = ['id', 'date'], inplace = True)

master['date'] = pd.to_datetime(master['date'])

master['year'] = master['date'].dt.year
master['week'] = master['date'].dt.week

master.drop(columns = 'd', inplace = True)

In [ ]:
# adding features and dropping columns in 'prices' for a more efficient merging
prices['id'] = prices['item'] + '_' + prices['store_code']
prices.drop(columns = ['item', 'store_code', 'category'], inplace = True)
prices

In [ ]:
# merging 'master' dataframe with 'prices'
master = pd.merge(
    master,
    prices,
    how = 'left',
    on = ['id', 'week', 'year']
)

master

In [ ]:
# filling in nulls in prices using backfill, as we want to get the last available correct price assuming it didn't change
master.sell_price.fillna(
    method = 'backfill',
    inplace = True
)

master

In [ ]:
# saving master dataframe
directory = '../data/features'
dfs = [
    { 'filename': 'master', 'df': master }
]

file_management.save_dfs_to_csv(dfs, directory)

## E. Creating aggregated dataframes <a class="anchor" id="master_df"></a>

In this section we will be generating several dataframes that we will be using throughout the different analysis.

In [17]:
# downloading the feature file from gdrive - in case you didn't run the previous section
directory = '../data/features/'
urls = [
    {'filename': 'master.csv', 'url': 'https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing'},
]
        
file_management.download_files_from_url(urls, directory)

master = pd.read_csv(directory + 'master.csv', index_col = 0)

master.csv file already exists in ../data/features/


### Preparing the master file

In [18]:
master['total_income'] = master['num_sales'] * master['sell_price']
master.drop(columns = ['weekday', 'year', 'week', 'weekday_int', 'event'], inplace = True)

### Global sales DF

In [19]:
sales_by_date = master.drop(columns = 'sell_price').groupby(['date']).agg('sum')
sales_by_date

,num_sales,total_income
date,,
2011-01-29,32631,"100,444.46"
2011-01-30,31749,"97,073.53"
2011-01-31,23783,"70,924.50"
2011-02-01,25412,"74,568.19"
2011-02-02,19146,"57,583.22"
...,...,...
2016-04-20,35343,"139,043.58"
2016-04-21,35033,"136,022.84"
2016-04-22,40517,"156,581.75"


In [ ]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_date', 'df': sales_by_date },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

### Global Sales by City

In [20]:
master['city'] = master['id'].apply(lambda x: x[-5:-2])
master

,id,num_sales,date,sell_price,total_income,city
0,ACCESORIES_1_001_BOS_1,0,2011-01-29,12.74,0.00,BOS
1,ACCESORIES_1_001_BOS_1,0,2011-01-30,12.74,0.00,BOS
2,ACCESORIES_1_001_BOS_1,0,2011-01-31,12.74,0.00,BOS
3,ACCESORIES_1_001_BOS_1,0,2011-02-01,12.74,0.00,BOS
4,ACCESORIES_1_001_BOS_1,0,2011-02-02,12.74,0.00,BOS
...,...,...,...,...,...,...
58327365,SUPERMARKET_3_827_PHI_3,0,2016-04-20,1.20,0.00,PHI
58327366,SUPERMARKET_3_827_PHI_3,0,2016-04-21,1.20,0.00,PHI
58327367,SUPERMARKET_3_827_PHI_3,0,2016-04-22,1.20,0.00,PHI
58327368,SUPERMARKET_3_827_PHI_3,0,2016-04-23,1.20,0.00,PHI


In [21]:
sales_by_date_city = master.drop(columns = 'sell_price').groupby(['date', 'city']).agg('sum')
sales_by_date_city

num_sales  total_income
date       city                         
2011-01-29 BOS        9438     29,632.47
           NYC       14195     44,581.95
           PHI        8998     26,230.04
2011-01-30 BOS        9630     29,226.48
           NYC       13805     43,017.43
...                    ...           ...
2016-04-23 NYC       21834     90,093.26
           PHI       14862     53,124.65
2016-04-24 BOS       12282     51,550.18
           NYC       23187     96,850.07
           PHI       14326     52,487.95

[5739 rows x 2 columns]

In [ ]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_date_city', 'df': sales_by_date_city },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

### Global Sales by Date and Store

In [22]:
master['store'] = master['id'].apply(lambda x: x[-5:])
master

,id,num_sales,date,sell_price,total_income,city,store
0,ACCESORIES_1_001_BOS_1,0,2011-01-29,12.74,0.00,BOS,BOS_1
1,ACCESORIES_1_001_BOS_1,0,2011-01-30,12.74,0.00,BOS,BOS_1
2,ACCESORIES_1_001_BOS_1,0,2011-01-31,12.74,0.00,BOS,BOS_1
3,ACCESORIES_1_001_BOS_1,0,2011-02-01,12.74,0.00,BOS,BOS_1
4,ACCESORIES_1_001_BOS_1,0,2011-02-02,12.74,0.00,BOS,BOS_1
...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_827_PHI_3,0,2016-04-20,1.20,0.00,PHI,PHI_3
58327366,SUPERMARKET_3_827_PHI_3,0,2016-04-21,1.20,0.00,PHI,PHI_3
58327367,SUPERMARKET_3_827_PHI_3,0,2016-04-22,1.20,0.00,PHI,PHI_3
58327368,SUPERMARKET_3_827_PHI_3,0,2016-04-23,1.20,0.00,PHI,PHI_3


In [23]:
sales_by_date_store = master.drop(columns = ['sell_price', 'city']).groupby(['date', 'store']).agg('sum')
sales_by_date_store

num_sales  total_income
date       store                         
2011-01-29 BOS_1       2556      8,096.63
           BOS_2       3852     12,204.35
           BOS_3       3030      9,331.49
           NYC_1       4337     13,414.01
           NYC_2       3494     11,223.84
...                     ...           ...
2016-04-24 NYC_3       7721     33,349.16
           NYC_4       3271     13,530.24
           PHI_1       4874     18,282.33
           PHI_2       5127     18,397.02
           PHI_3       4325     15,808.59

[19130 rows x 2 columns]

In [ ]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_date_store', 'df': sales_by_date_store },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

### Sales per Item

In [5]:
# downloading the feature file from gdrive - in case you didn't run the previous section
directory = '../data/features/'
urls = [
    {'filename': 'master.csv', 'url': 'https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing'},
]
        
file_management.download_files_from_url(urls, directory)

master = pd.read_csv(directory + 'master.csv', index_col = 0)

master.csv file already exists in ../data/features/


In [6]:
master.head()

,id,num_sales,date,weekday,weekday_int,event,year,week,sell_price
0,ACCESORIES_1_001_BOS_1,0,2011-01-29,Saturday,1,None,2011,4,12.74
1,ACCESORIES_1_001_BOS_1,0,2011-01-30,Sunday,2,None,2011,4,12.74
2,ACCESORIES_1_001_BOS_1,0,2011-01-31,Monday,3,None,2011,5,12.74
3,ACCESORIES_1_001_BOS_1,0,2011-02-01,Tuesday,4,None,2011,5,12.74
4,ACCESORIES_1_001_BOS_1,0,2011-02-02,Wednesday,5,None,2011,5,12.74


In [7]:
master.drop(columns = ['date', 'weekday', 'weekday_int', 'event', 'year', 'week' ], inplace = True)

In [8]:
master['item'] = master['id'].apply(lambda x: x[:-6])
master

,id,num_sales,sell_price,item
0,ACCESORIES_1_001_BOS_1,0,12.74,ACCESORIES_1_001
1,ACCESORIES_1_001_BOS_1,0,12.74,ACCESORIES_1_001
2,ACCESORIES_1_001_BOS_1,0,12.74,ACCESORIES_1_001
3,ACCESORIES_1_001_BOS_1,0,12.74,ACCESORIES_1_001
4,ACCESORIES_1_001_BOS_1,0,12.74,ACCESORIES_1_001
...,...,...,...,...
58327365,SUPERMARKET_3_827_PHI_3,0,1.20,SUPERMARKET_3_827
58327366,SUPERMARKET_3_827_PHI_3,0,1.20,SUPERMARKET_3_827
58327367,SUPERMARKET_3_827_PHI_3,0,1.20,SUPERMARKET_3_827
58327368,SUPERMARKET_3_827_PHI_3,0,1.20,SUPERMARKET_3_827


In [13]:
sales_by_product = master.drop(columns = ['id']).groupby(['item']).agg([np.sum, np.mean, np.std])
sales_by_product

num_sales           sell_price           
                        sum mean  std        sum  mean  std
item                                                       
ACCESORIES_1_001       4093 0.21 0.58 219,344.58 11.47 0.73
ACCESORIES_1_002       5059 0.26 0.59 100,941.36  5.28 0.09
ACCESORIES_1_003       1435 0.08 0.32  75,518.03  3.95 0.13
ACCESORIES_1_004      39175 2.05 2.67 114,391.52  5.98 0.28
ACCESORIES_1_005      14621 0.76 1.23  73,414.40  3.84 0.22
...                     ...  ...  ...        ...   ...  ...
SUPERMARKET_3_823     15388 0.80 1.71  63,984.10  3.34 0.23
SUPERMARKET_3_824      8325 0.44 0.95  57,897.00  3.03 0.26
SUPERMARKET_3_825     13526 0.71 1.20  94,370.36  4.93 0.24
SUPERMARKET_3_826     12188 0.64 1.25  29,381.33  1.54 0.01
SUPERMARKET_3_827     11377 0.59 2.03  22,951.80  1.20 0.01

[3049 rows x 6 columns]

In [14]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_product', 'df': sales_by_product },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

./../data/features/sales_by_product.csv doesn't exist. Creating new file


['./../data/features/sales_by_product.csv']

### Sales per Store

In [14]:
# downloading the feature file from gdrive - in case you didn't run the previous section
directory = '../data/features/'
urls = [
    {'filename': 'master.csv', 'url': 'https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing'},
]
        
file_management.download_files_from_url(urls, directory)

master = pd.read_csv(directory + 'master.csv', index_col = 0)

master.csv file already exists in ../data/features/


In [10]:
master['store'] = master['id'].apply(lambda x: x[-5:])
master['total_income'] = master['num_sales'] * master['sell_price']
master.drop(columns = ['weekday', 'year', 'week', 'weekday_int', 'event', 'date', 'id'], inplace = True)

In [11]:
master.head(1)

,num_sales,sell_price,store,total_income
0,0,12.74,BOS_1,0.00


In [12]:
sales_by_store = master.groupby(['store']).agg(['count', 'sum', np.mean, 'std'])
sales_by_store

num_sales                     sell_price                          \
          count       sum mean  std      count           sum mean  std   
store                                                                    
BOS_1   5832737   5595292 0.96 3.33    5832737 32,235,235.37 5.53 4.52   
BOS_2   5832737   7214384 1.24 4.42    5832737 32,179,182.33 5.52 4.50   
BOS_3   5832737   6089330 1.04 3.80    5832737 32,329,719.52 5.54 4.55   
NYC_1   5832737   7698216 1.32 4.06    5832737 32,525,656.93 5.58 4.55   
NYC_2   5832737   5685475 0.97 2.76    5832737 32,554,083.34 5.58 4.56   
NYC_3   5832737  11188180 1.92 6.21    5832737 32,334,050.18 5.54 4.55   
NYC_4   5832737   4103676 0.70 2.00    5832737 32,523,875.04 5.58 4.55   
PHI_1   5832737   5149062 0.88 2.42    5832737 32,599,974.02 5.59 4.56   
PHI_2   5832737   6544012 1.12 3.87    5832737 32,696,891.78 5.61 4.59   
PHI_3   5832737   6427782 1.10 4.07    5832737 32,553,972.67 5.58 4.56   

      total_income                           
             count           sum mean   std  
store                                        
BOS_1      5832737 19,340,893.82 3.32  9.61  
BOS_2      5832737 25,266,780.37 4.33 12.71  
BOS_3      5832737 21,946,513.55 3.76 13.99  
NYC_1      5832737 27,735,269.87 4.76 13.06  
NYC_2      5832737 21,507,127.31 3.69  9.26  
NYC_3      5832737 39,492,258.61 6.77 18.11  
NYC_4      5832737 15,046,818.84 2.58  6.82  
PHI_1      5832737 18,235,243.77 3.13  7.71  
PHI_2      5832737 21,658,283.67 3.71 11.57  
PHI_3      5832737 20,752,293.45 3.56 10.67

In [13]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_store', 'df': sales_by_store },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

sales_by_store file already exists in ../data/features
Overwriting file


['./../data/features/sales_by_store.csv']

### Sales per Store and Item

In [47]:
# downloading the feature file from gdrive - in case you didn't run the previous section
directory = '../data/features/'
urls = [
    {'filename': 'master.csv', 'url': 'https://drive.google.com/file/d/1_OCpE6AZK3ju5RJVTJZm8ox5Cn8G3_Ag/view?usp=sharing'},
]
        
file_management.download_files_from_url(urls, directory)

master = pd.read_csv(directory + 'master.csv', index_col = 0)

master.csv file already exists in ../data/features/


In [48]:
master.drop(columns = ['weekday', 'weekday_int', 'event', 'year', 'week', 'sell_price'], inplace = True)
master['store'] = master['id'].apply(lambda x: x[-5:])
master['item'] = master['id'].apply(lambda x: x[:-6])
master.drop(columns = 'id', inplace = True)

In [49]:
master

,num_sales,date,store,item
0,0,2011-01-29,BOS_1,ACCESORIES_1_001
1,0,2011-01-30,BOS_1,ACCESORIES_1_001
2,0,2011-01-31,BOS_1,ACCESORIES_1_001
3,0,2011-02-01,BOS_1,ACCESORIES_1_001
4,0,2011-02-02,BOS_1,ACCESORIES_1_001
...,...,...,...,...
58327365,0,2016-04-20,PHI_3,SUPERMARKET_3_827
58327366,0,2016-04-21,PHI_3,SUPERMARKET_3_827
58327367,0,2016-04-22,PHI_3,SUPERMARKET_3_827
58327368,0,2016-04-23,PHI_3,SUPERMARKET_3_827


In [51]:
sales_by_date_store_item = master.groupby(['date', 'store', 'item']).sum()
sales_by_date_store_item

num_sales
date       store item                        
2011-01-29 BOS_1 ACCESORIES_1_001           0
                 ACCESORIES_1_002           0
                 ACCESORIES_1_003           0
                 ACCESORIES_1_004           1
                 ACCESORIES_1_005           0
...                                       ...
2016-04-24 PHI_3 SUPERMARKET_3_823          1
                 SUPERMARKET_3_824          0
                 SUPERMARKET_3_825          0
                 SUPERMARKET_3_826          3
                 SUPERMARKET_3_827          0

[58327370 rows x 1 columns]

In [52]:
directory = '../data/features'
dfs = [
    { 'filename': 'sales_by_date_store_item', 'df': sales_by_date_store_item },
]

file_management.save_dfs_to_csv(dfs, directory, prefix = '')

sales_by_date_store_item file already exists in ../data/features
Overwriting file


['./../data/features/sales_by_date_store_item.csv']